# Assignment P1: Sentiment Ananlysis
This notebook illustrates the Assignment P1 of CSC 791 Natural Language Processing Fall 2020. In this assignment you will learn how to use word vectors to classify sentences based on the sentiment they express Specifically, this assignment emphasizes long sentences, which are often challenging for existing sentiment analysis tools.


In [ ]:
#Download Spacy 
!python -m spacy download en_core_web_md
!python3 -m spacy download en_core_web_sm
!python -m spacy.en.download all

In [ ]:
# Upload Testing and Training datasets
# Please select both the files: P1_training and P1_testing

from google.colab import files
uploaded = files.upload()

Saving P1_testing.xlsx to P1_testing.xlsx
Saving P1_training.xlsx to P1_training.xlsx


In [ ]:
import pandas as pd
import numpy as np
import io

In [ ]:
# Load Training and Testing data
df_train = pd.read_excel(io.BytesIO(uploaded['P1_training.xlsx']))
df_test  = pd.read_excel(io.BytesIO(uploaded['P1_testing.xlsx']))

In [ ]:
print(df_train.shape)
print(df_test.shape)

(1660, 2)
(683, 2)


In [ ]:
# Analyzing the dataset
# df_train.head()
df_train.groupby(by = ["label"]).count()

,sentence
label,
0,263
1,736
2,661


In [ ]:
# Import
import spacy
from spacy.lang.en import English

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding

# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
from sklearn.metrics import precision_recall_fscore_support as pr

In [ ]:
# Load Spacy model
import en_core_web_sm
# nlp = en_core_web_sm.load()
nlp= spacy.blank('en')
nlp

In [ ]:
# Stop words
from  spacy.lang.en.stop_words import STOP_WORDS

stop_words = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
# Temporary splitting of testing and training data
from sklearn.model_selection import train_test_split

# X = df_train['sentence']
# Y = df_train['label']
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train, Y_train = df_train['sentence'], df_train['label']
X_test,  Y_test  = df_test['sentence'], df_test['label']

X_train = list(X_train)
X_test  = list(X_test)
Y_train = list(Y_train)
Y_test  = list(Y_test)

In [ ]:
def accuracy(Y_test, prediction):
  same = 0 
  for i,j in zip(Y_test, prediction):
    if i==j:
      same += 1
  return (same/len(Y_test))

# Baseline Method 1:
Using a combination of TFIDF vectorizer and Linear SVM (Linear Support Vector Classification) to classify the sentence sentiments. TF-IDF is an improvement to the pre-existing count vectorizer where the count of occurrences of each word in the document is taken and the words which have lower frequency and higner uniquness are given higher weightage. 

###Reference:
* https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
* https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from spacy.tokenizer import Tokenizer

In [ ]:
parser = English()

# Tokenizer funtion

def spacy_tokenizer(sentence):
    tokens = parser(sentence)

    # Lemmatizing and Stop Word removal
    tokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens ]
    tokens = [ word for word in tokens if word not in stop_words]

    # return preprocessed list of tokens
    return tokens


In [ ]:
# Vectorization and Classifier definition

vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1)) 
classifier = LinearSVC()

In [ ]:
# Transformer 
# Reference: https://spacy.io/usage/training

class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

def clean_text(text):     
    return text.strip().lower()


In [ ]:
# Count Vectorizer code:

# # Pipeline to Clean, Tokenize, Vectorize, and Classify

# pipeline_baseline_1 = Pipeline([ ("cleaner", predictors()),
#                                 ('vectorizer', vectorizer),
#                                 ('classifier', classifier)])

# # Fit
# pipeline_baseline_1.fit(X_train, Y_train)

# # Prediction
# prediction = pipeline_baseline_1.predict(X_test)

# # Labels
# # 0, 1, 2 => positive, neutral or negative
    
# from sklearn.metrics import classification_report, confusion_matrix

# print(confusion_matrix(Y_test, prediction))
# print(classification_report(Y_test, prediction))

In [ ]:
vectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1)) 
vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function spacy_tokenizer at 0x7fce3eae2378>,
                use_idf=True, vocabulary=None)

In [ ]:
# Pipeline to Clean, Tokenize, Vectorize, and Classify
pipeline_baseline_2 = Pipeline([ ("cleaner", predictors()),
                                ('vectorizer', vectorizer),
                                ('classifier', classifier)])

# Fit
pipeline_baseline_2.fit(X_train, Y_train)

# Prediction
prediction = pipeline_baseline_2.predict(X_test)

# Labels
# 0, 1, 2 => positive, neutral or negative

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(Y_test, prediction))
print(classification_report(Y_test, prediction))
print("Accuracy: ", accuracy(Y_test, prediction))

[[  8  41  33]
 [  4 203  96]
 [ 13 108 177]]
              precision    recall  f1-score   support

           0       0.32      0.10      0.15        82
           1       0.58      0.67      0.62       303
           2       0.58      0.59      0.59       298

    accuracy                           0.57       683
   macro avg       0.49      0.45      0.45       683
weighted avg       0.55      0.57      0.55       683

Accuracy:  0.568081991215227


# Baseline Method 2:

TensorFlow Universal Sentence Encoder:
Using the tensorflow sentence encoder after basic pre processing

Reference: https://tfhub.dev/google/universal-sentence-encoder-lite/2



In [ ]:
!pip3 install sentencepiece
import sentencepiece as spm
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


In [ ]:
df_train = pd.read_excel(io.BytesIO(uploaded['P1_training.xlsx']))
df_test  = pd.read_excel(io.BytesIO(uploaded['P1_testing.xlsx']))

In [ ]:
# Refernce: https://tfhub.dev/google/universal-sentence-encoder-lite/2

with tf.Session() as sess:
  module = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-lite/2")
  spm_path = sess.run(module(signature="spm_path"))

sp = spm.SentencePieceProcessor()
sp.Load(spm_path)

input_placeholder = tf.sparse_placeholder(tf.int64, shape=[None, None])
encodings = module(
    inputs=dict(
        values=input_placeholder.values,
        indices=input_placeholder.indices,
        dense_shape=input_placeholder.dense_shape))

In [ ]:
def process_to_IDs_in_sparse_format(sp, sentences):
  # An utility method that processes sentences with the sentence piece processor
  # 'sp' and returns the results in tf.SparseTensor-similar format:
  # (values, indices, dense_shape)

  ids = [sp.EncodeAsIds(x) for x in sentences]
  max_len = max(len(x) for x in ids)
  dense_shape=(len(ids), max_len)
  values=[item for sublist in ids for item in sublist]
  indices=[[row,col] for row in range(len(ids)) for col in range(len(ids[row]))]
  return (values, indices, dense_shape)

In [ ]:
# Testing out the pre trained model
# Refrence: https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder_lite.ipynb

tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])

  values, indices, dense_shape = process_to_IDs_in_sparse_format(sp, df_train['sentence'].values.tolist())
  message_embeddings = session.run(
      encodings,
      feed_dict={input_placeholder.values: values,
                input_placeholder.indices: indices,
                input_placeholder.dense_shape: dense_shape})
  df_train['vector'] = np.array(message_embeddings).tolist()
  
  values, indices, dense_shape = process_to_IDs_in_sparse_format(sp, df_test['sentence'].values.tolist())
  message_embeddings = session.run(
      encodings,
      feed_dict={input_placeholder.values: values,
                input_placeholder.indices: indices,
                input_placeholder.dense_shape: dense_shape})
  df_test['vector'] = np.array(message_embeddings).tolist()


In [ ]:
df_train['tf_vector'] = df_train.apply(lambda i: i['vector'], axis=1)
df_test['tf_vector']  = df_test.apply(lambda  i: i['vector'], axis=1)

In [ ]:
# Trasforming vector info

X_train = np.array(df_train['tf_vector'].values.tolist())
Y_train = np.array(df_train['label'].values.tolist())

X_test  = np.array(df_test['tf_vector'].values.tolist())
Y_test  = np.array(df_test['label'].values.tolist())


In [ ]:
# Classifier

classifier = LinearSVC()
classifier.fit(X_train, Y_train)

prediction = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(Y_test, prediction))
print(classification_report(Y_test, prediction))
print("Accuracy:", accuracy(Y_test, prediction))

[[ 13  46  23]
 [ 18 212  73]
 [ 15  93 190]]
              precision    recall  f1-score   support

           0       0.28      0.16      0.20        82
           1       0.60      0.70      0.65       303
           2       0.66      0.64      0.65       298

    accuracy                           0.61       683
   macro avg       0.52      0.50      0.50       683
weighted avg       0.59      0.61      0.60       683

Accuracy: 0.6076134699853587


# Proposed Method:
Using the default spacy vectorizer to classify the sentence sentiments. Spacy is an open-source Python framework for NLP. Spacy comes with a pre-trained word2vec model with a vocabulary of more than a million words. The new approach introduced is averaging out the vectors from individual tokens generated by the Spacy tokenizer.

Reference: 
https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/


In [ ]:
# Load Spacy model
import spacy.cli

spacy.cli.download("en_core_web_lg")
nlp = spacy.load('en_core_web_lg')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
# For Spacy word2vec

def createVector(sent):
  vect = nlp(sent['sentence'])
  avg = np.zeros(300)
  # print(vect)
  
  # Calculate average vector
  for token in vect: 
    avg += token.vector
  vect_avg = avg/len(vect)

  return vect_avg.tolist()

In [ ]:
# Apply Transformation
df_train['vector'] = df_train.apply(lambda i : createVector(i), axis=1)
df_test['vector']  = df_test.apply(lambda i  : createVector(i), axis=1)

In [ ]:
# Trasforming Vector info

X_train = np.array(df_train['vector'].values.tolist())
Y_train = np.array(df_train['label'].values.tolist())

X_test  = np.array(df_test['vector'].values.tolist())
Y_test  = np.array(df_test['label'].values.tolist())


In [ ]:
# Classifier
classifier = LinearSVC()
classifier.fit(X_train,Y_train)

prediction = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(Y_test, prediction))
print(classification_report(Y_test, prediction))
print("Accuracy:", accuracy(Y_test, prediction))

[[  7  45  30]
 [ 21 207  75]
 [ 11  87 200]]
              precision    recall  f1-score   support

           0       0.18      0.09      0.12        82
           1       0.61      0.68      0.64       303
           2       0.66      0.67      0.66       298

    accuracy                           0.61       683
   macro avg       0.48      0.48      0.47       683
weighted avg       0.58      0.61      0.59       683

Accuracy: 0.6061493411420205
